# REDES NEURONALES PROFUNDAS
Utilice el conjunto de datos Fashion-MNIST para construir un clasificador convolucional de imágenes de productos. Para la construcción del modelo utilice los dos esquemas que se describen a continuación y compare los resultados:

1. Entrenar de un Autocodificador convolucional multicapa.
2. Extraer la reducción de la dimensionalidad que el Autocodificador construye en el entrenamiento.

Para los puntos 2) compruebe, y muestre con ejemplos, que las imágenes están bien reconstruidas.

## OBJETIVO
Aplicar el proceso de aprendizaje a partir de datos para resolver problemas de clasificación utilizando redes neuronales convolucionales profundas sobre la herramienta Keras.

## DATOS
Incluidos en Keras.
También, existe otra fuente equivalente que se consigue en el siguiente URL https://www.kaggle.com/zalando-research/fashionmnist donde hay un resumen de estos datos en el archivo CVS y XLSX.

la clasificación para el aprendizaje supervisado es:

    Label 	Class
    0 	 	T-shirt/top
    1 	 	Trouser
    2 	 	Pullover
    3 	 	Dress
    4 	 	Coat
    5 	 	Sandal
    6 	 	Shirt
    7 	 	Sneaker
    8 	 	Bag
    9 	 	Ankle boot

**Importante: Lea los comentarios y apuntes del Notebook para tener claridad de los pasos.**
### Consideraciones
- Utilice sólo los conjuntos de datos indicado.
- El frameworks a utilizar es TensorFlow, Keras con Jupyter Notebbooks.

### Enlaces de interés
- documentación Keras, URL: https://keras.io/models/sequential/
- documentación TensorFlow, URL: https://www.tensorflow.org/versions
- Tutorial CNN basico, URL: https://www.kaggle.com/nhlr21/deep-keras-cnn-tutorial/notebook


## Importando Librerias

In [ ]:
# importando dependencias de trabajo
# importando librerias basicas
import os
import sys
import re
import gc

# importando modulos de analisis de datos, ML y graficas
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow import keras
from collections import OrderedDict
from collections import Counter

# importando dependencias para tensorflow
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# importando para sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline

# importando para keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import UpSampling2D
from keras.layers import BatchNormalization
from keras.layers import TimeDistributed

## Funciones Utiles

In [ ]:
# funcion que recibe una lista numpy y recupera la forma de cada elemento, devuelve una lista con formas
def get_shape(data):
    # respuesta de la funcion
    ans = list()

    for d in data:
        sp = d.shape
        ans.append(sp)
    return ans

In [ ]:
# funcion que transforma el entero de la clase a la palabra de la etiqueta, devuelve una lista de etiquetas
def class2label(data, labels):
    # respuesta de la funcion
    ans = list()

    for d in data:
        d = int(d)
        l = str(labels[d])
        ans.append(l)
    return ans

In [ ]:
# funcion que estandariza los datos en numpy de acuerdo a un valor min & max, devuelve un arreglo np flotante
def std_data(data, minv, maxv):
    rangev = maxv - minv
    ans = data.astype("float32")/float(rangev)
    # ans = pd.Series(ans)
    # respuesta de la funcion
    return ans

## Cargar y Preparar los Datos

Los pasos de esta seccion son:

1. Leer los datos desde MNIST.
2. Formatear los datos para que los acepte el DataFrame de Pandas.
2. Crear el DataFrame de Pandas con un esquema propio.
2. Formatear los datos MNIST para pobrar el DataFrame de pandas.
3. Revisar que todo este como deberia estar.

In [ ]:
# lista de nombres de las clasificaciones
label_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
# se carga el archivo de datos de trabajo por medio de Keras
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
# nombres de columnas para el dataframe de pandas
col_names = ["img_data", "img_shape", "class", "label", "std_img_data", "cat_labels"]#, "ReshapeData", "Label", "Class", "DataSize", "ReshapeSize", "ResKeras", "ScoreKeras"]
# creando dataframe con columnas
fashion_df = pd.DataFrame(columns=col_names)

In [ ]:
# integrando datos de mnist
img_data = np.concatenate((x_train, x_test), axis = 0)
class_data = np.concatenate((y_train, y_test), axis = 0)
# recuperando forma de imagenes
img_shape = get_shape(img_data)
# recuperando etiquetas de las clases
labels = class2label(class_data, label_names)
# estandarizar los datos de la imagen
std_img_data = std_data(img_data, 0, 255)
# categorizando las clases a aprender
cat_labels = to_categorical(class_data, len(label_names))

In [ ]:
# cambio de formato para utilizar el dataframe
img_data = img_data.tolist()
std_img_data = std_img_data.tolist()
cat_labels = cat_labels.tolist()

In [ ]:
# definir arreglo basico de datos
data_list = (img_data, img_shape, class_data, labels, std_img_data, cat_labels)

In [ ]:
# poblamdo las columnas del dataframe
for col, data in zip(col_names, data_list):
    fashion_df[col] = data

In [ ]:
fashion_df.info()

In [ ]:
fashion_df

In [ ]:
# libero memoria
gc.collect()

## Preprocesar los Datos

Los pasos de esta seccion son:

1. Revisar que los datos esten bien.
2. Elegir la caracteristicas o propiedades de aprendizaje.
3. Elegir la variable objetivo del aprendizaje.
4. Dividir la conjunto de datos entre las poblaciones de entrenamiento y pruebas.
5. Formatear los datos de aprendizaje y objetivo acorde a la red neuronal.

In [ ]:
# cchequeo la distribucion de datos
sns.set()
plt.figure(figsize=(20, 8))
sns.histplot(fashion_df[col_names[3]])
plt.show()

In [ ]:
# seleccionando caracteristicas de aprendizaje y variables objetivo
# recuperando la forma de las imagenes basado en el primer elemento de la lista

# recuperando los valores y ajustando el tensor para la CNN
A = fashion_df[col_names[4]]
# recuperando los valores de la cateogoria
b = fashion_df[col_names[5]].values

# fortateo de datos numpy
X = np.array([np.array(i, dtype="object") for i in A], dtype="object")
y = np.array([np.array(j, dtype="object") for j in b], dtype="object")

print(X.shape)
# forma basica general de las imagenes
imgsh = X[0].shape
# ajuste de forma para el modelo CNN
X = X.reshape(fashion_df.shape[0], imgsh[0], imgsh[1], 1)
print(X.shape)

In [ ]:
# semilla para el random
rseed = 42

# en tamanho de la muestra para pruebas esta entre 0.2 y 0.3
train_pop = 0.8
test_pop = 1.0 - train_pop

In [ ]:
# distribuir los datos entre entrenamiento vs. pruebas
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_pop, random_state = rseed)

In [ ]:
# formateo para keras y tensorflow
X_train = tf.convert_to_tensor(X_train, dtype="float64")
y_train = tf.convert_to_tensor(y_train, dtype="float64")
X_test = tf.convert_to_tensor(X_test, dtype="float64")
y_test = tf.convert_to_tensor(y_test, dtype="float64")

## Definir Modelo CNN

Los pasos de esta seccion son:

1. Definir las variables topologicas de la red neuronal.
2. Definir los parametros de optimizacion y aprendizaje del modelo.
3. Definir la topologia (capas) del modelo.
4. Definir las condiciones de entrenamiento para el modelo.

In [ ]:
# defino parametros necesarios para el modelo Autoencoder
# parametros para las capas
mid_neurons = 16*16
filters = 16
outn = len(label_names)
ksize = (3,3)
psize = (2,2)
act = "relu"
out = "sigmoid"
pad = "same"
ldrop = 0.2

# forma del kernel de entrada y de las capas intermedias
inshape = X[0].shape

# parametros de optimizacion del modelo
l = "categorical_crossentropy"
opti = "adam"
met = ["accuracy"]

# parametros de operacion/aprendizaje del modelo
ver = 1
epo = 50
bs = 32

In [ ]:
# definicion de las capas para el Autoencoder
layers = (
    # capa de entrada
    Input(shape = inshape, name = "LayIn"),
    # capa convolucional intermedia con regularizacion
    Conv2D(filters, ksize, activation = act, padding = pad, name = "EnConv1"),
    MaxPooling2D(psize, padding = pad, name = "EnPool1"),
    # BatchNormalization(name = "EnNorm1"),
    Dropout(ldrop, name = "EnDrop1"),

    # capa convolucional intermedia con regularizacion
    Conv2D(int(filters)/2, ksize, activation=act, padding = pad, name = "EnConv2"),
    MaxPooling2D(psize, padding = pad, name = "EnPool2"),
    # BatchNormalization(name = "EnNorm2"),
    # Dropout(ldrop, name = "EnDrop2"),
    
    # capa intermedia de 2D a 1D
    Flatten(name = "LayFlat"),
    # # # capa intermedia densamente poblada con regularizacion
    Dense(mid_neurons, activation = act, name = "DenseMid"),
    Dropout(ldrop, name = "MidDrop"),
    # # # capa intermedia de 1D a 2D
    
    # capa densamente poblada para clasificar los datos 
    Dense(int(mid_neurons)/2, activation = act, name = "DenseClass1"),
    Dropout(ldrop, name = "ClsDrop1"),

    # capa densamente poblada para clasificar los datos 
    Dense(int(mid_neurons)/4, activation = act, name = "DenseClass2"),
    Dropout(ldrop, name = "ClsDrop2"),

    # capa de salida
    Dense(outn, activation=out, name = "LayOut")
)

In [ ]:
# definiendo el modelo CNN en Keras
cnn_model = Sequential(layers)
cnn_model.model_name = "DCNN Classifier"

In [ ]:
# compilando las condiciones de optimizacion y ajuste del Modelo CNN
cnn_model.compile(loss = l, optimizer = opti, metrics = met)

In [ ]:
# resumen de la topologia del modelo
cnn_model.summary()

In [ ]:
# condiciones de parada temprana
cnn_earlystop_acc = EarlyStopping(monitor = "val_accuracy", min_delta = 0.001, patience = 7, verbose = ver, mode = "max", restore_best_weights = True)

## Entrenar Modelo

Los pasos de esta seccion son:

1. Entrenar el modelo con el conjunto de entrenamineto.

In [ ]:
# ajustando el modelo MLP Keras
cnn_log = cnn_model.fit(
    x = X_train,#np.array(X_trainB), 
    y = y_train,#to_categorical(np.array(y_trainB), categories), 
    batch_size = bs,
    epochs = epo, 
    verbose = ver,
    callbacks = [cnn_earlystop_acc],
    workers = 8,
    shuffle = False,
    use_multiprocessing = True,
    validation_data = (X_test, y_test)
)

## Probar Modelo

Los pasos de esta seccion son:

1. Probar el modelo con el conjunto de pruebas.
2. Evaluar globalmente los resultados.
3. Guardar el modelo entrenado.

In [ ]:
cnn_eval = cnn_model.evaluate(x = X_test, y = y_test)

In [ ]:
# resultados generales
print("Perdida promedio: ", cnn_eval[0])
print("Precision promedio: ", cnn_eval[1])

In [ ]:
# pruebas sobre el modelo
cnn_predictions = cnn_model.predict(X_test, verbose = ver)

In [ ]:
# guardar el modelo entrenado
wdir = os.getcwd()
folder_models = "Models"
model_fname = "hdig_cnn_classifier"
model_fpn = os.path.join(folder_models, model_fname)
print("El modelo entrenado esta en:", model_fpn)
cnn_model.save(model_fpn)
# tf.keras.models.save_model(cnn_autoencoder, model_fpn)

## Mostrar Resultados

Los pasos de esta sección son:

1. Mostrar las curvas de aprendizaje.
2. Mostrar la Matriz de confusión del Clasificador.
3. Mostrar la clasificación del modelo.
4. Mostrar la abstracción del Clasificador.

In [ ]:
# ajuste de las predicciones para ver el reporte de matrix de confusion
cnn_predictions = np.array(cnn_predictions).argmax(axis=1)

In [ ]:
# Informe por consola de las pruebas para el clasificador
print("----- Reporte de Pruebas para el clasificador CNN -----")
print("--- Conteo ---\n" + str(Counter(cnn_predictions)))
print("--- Matriz de Confusion ---\n" + str(confusion_matrix(y_test, cnn_predictions)))
print("--- Reporte de Pruebas: ---")
print(classification_report(y_test, cnn_predictions))
print("--- Puntaje General---\n")
print(" - Perdida: ", cnn_eval[0])
print(" - Precision: ", cnn_eval[1])

In [ ]:
# reporte del aprendizaje
# base de la figura
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (16,8))

# datos de la figura en de perdida y precision
ax1.plot(cnn_log.history["loss"], 'red', label = "Train Loss")
ax1.plot(cnn_log.history["val_loss"], 'darkorange', label = "Test Loss")
ax2.plot(cnn_log.history["accuracy"], 'red', label = "Train Accuracy")
ax2.plot(cnn_log.history["val_accuracy"], 'royalblue', label = "Test Accuracy")

# leyenda de la grafica
fig.suptitle("LEARNING BEHAVIOR")
ax1.grid(True)
ax2.grid(True)
ax1.set_title("Loss")
ax2.set_title("Accuracy")
ax1.set(xlabel = "Epoch [cycle]", ylabel = "loss [%]")
ax2.set(xlabel = "Epoch [cycle]", ylabel = "Acc [%]")
fig.legend()
fig.show()

In [ ]:
# prueba funcional del autoencoder
max_img = 10
random_test_img = np.random.randint(len(X_test), size = max_img)

In [ ]:
middle_layer = "LayFlat"
cnn_abstraction = Model(inputs = cnn_autoencoder.input, outputs = cnn_autoencoder.get_layer(middle_layer).output)

In [ ]:
# desplegando pruebas
plt.figure(figsize=(20, 4))
og_shape = fashion_df[col_names[1]][0]

for i, img_id in enumerate(random_test_img):
    # imagen original
    ax = plt.subplot(3, max_img, i + 1)
    temp_X = np.array(X_test[img_id])
    plt.imshow(temp_X.reshape(og_shape))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # imagen abstracta
    ax = plt.subplot(3, max_img, max_img + i + 1)
    temp_abstract = cnn_abstraction(temp_X)
    temp_abstract = np.array(temp_abstract)
    plt.imshow(temp_abstract)
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # imagen reconstruida
    ax = plt.subplot(3, max_img, 2*max_img + i + 1)
    temp_pre = np.array(cnn_predictions[img_id])
    plt.imshow(temp_pre.reshape(og_shape))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()